### Import TFRS

First, install and import TFRS:

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.1 MB/s eta 0:00:00


In [1]:
from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

### Read the data

In [ ]:
# # Ratings data.
# ratings = tfds.load('movielens/100k-ratings', split="train")
# # Features of all the available movies.
# movies = tfds.load('movielens/100k-movies', split="train")

# # Select the basic features.
# ratings = ratings.map(lambda x: {
#     "movie_title": x["movie_title"],
#     "user_id": x["user_id"]
# })
# movies = movies.map(lambda x: x["movie_title"])

In [5]:
# Ratings data.
ratings = pd.read_csv('../Downloads/user_rating.csv')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))
# Features of all the available movies.
movies = pd.read_csv('../Downloads/recipe_dataset.csv')
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
movies = movies.map(lambda x: x["Title"])

Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [6]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
# user_ids_vocabulary.adapt(ratings.map(lambda x: tf.strings.as_string(x["user_id"])))
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [7]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

Define the two models and the retrieval task.

In [8]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [9]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
6/6 [==============================] - 4s 523ms/step - factorized_top_k/top_1_categorical_accuracy: 4.5314e-05 - factorized_top_k/top_5_categorical_accuracy: 9.0629e-05 - factorized_top_k/top_10_categorical_accuracy: 9.0629e-05 - factorized_top_k/top_50_categorical_accuracy: 9.9692e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0033 - loss: 27679.5469 - regularization_loss: 0.0000e+00 - total_loss: 27679.5469
Epoch 2/3
6/6 [==============================] - 3s 538ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0141 - factorized_top_k/top_5_categorical_accuracy: 0.0298 - factorized_top_k/top_10_categorical_accuracy: 0.0520 - factorized_top_k/top_50_categorical_accuracy: 0.1280 - factorized_top_k/top_100_categorical_accuracy: 0.1797 - loss: 25341.3809 - regularization_loss: 0.0000e+00 - total_loss: 25341.3809
Epoch 3/3
6/6 [==============================] - 3s 525ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0397 - factorized_top_k/top_5_categorical_

In [10]:
# Get some recommendations.
_, titles = index(np.array(["0"]))
print(f"Top 3 recommendations for user 1551: {titles[0, :3]}")

Top 3 recommendations for user 1551: [b'Tahu & Mie Crispy' b'Sop kikil (kaki kambing)'
 b'Ikan Sarden Sambel Merah No MSG']


In [11]:
# Get some recommendations.
_, titles = index(np.array(["15051"]))
print(f"Top 3 recommendations for user 15051: {titles[0, :3]}")

Top 3 recommendations for user 15051: [b'Tahu & Mie Crispy' b'Sop kikil (kaki kambing)'
 b'Ikan Sarden Sambel Merah No MSG']


In [12]:
# Get some recommendations.
_, titles = index(np.array(["15052"]))
print(f"Top 3 recommendations for user 15051: {titles[0, :3]}")

Top 3 recommendations for user 15051: [b'Tahu & Mie Crispy' b'Sop kikil (kaki kambing)'
 b'Ikan Sarden Sambel Merah No MSG']


In [13]:
# Get some recommendations.
_, titles = index(np.array(["200"]))
print(f"Top 3 recommendations for user 1551: {titles[0, :3]}")

Top 3 recommendations for user 1551: [b'Tumis Cumi asin pedas campur tahu..' b'Thai Beef Salad'
 b'Bubur Ayam Bumbu Kuning']


In [14]:
# Get some recommendations.
_, titles = index(np.array(["100"]))
print(f"Top 3 recommendations for user 15051: {titles[0, :3]}")

Top 3 recommendations for user 15051: [b'Sate Kambing Empuk Sambel Kecap' b'Bola Bola isi tahu'
 b'Tempe Kemangi Bu-Ris Enaak']


In [15]:
# Get some recommendations.
_, titles = index(np.array(["1000"]))
print(f"Top 3 recommendations for user 15051: {titles[0, :3]}")

Top 3 recommendations for user 15051: [b'Kari telur tempe tahu' b'Chicken katsu' b'Chicken katsu']


In [16]:
# Get some recommendations.
_, titles = index(np.array(["150"]))
print(f"Top 3 recommendations for user 15051: {titles[0, :3]}")

Top 3 recommendations for user 15051: [b'Asam pedas ceker tulang ayam kuah mercon' b'Bandeng Bakar tulang lunak'
 b'Ca brokoli ayam']


## **Calculate validation dataset**

In [17]:
# Ratings data.
ratings = pd.read_csv('user_rating.csv')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

# Split into train and test set in rating dataset
tf.random.set_seed(42)
shuffled = ratings.shuffle(len(ratings), seed=42, reshuffle_each_iteration=False)
ratings_train = shuffled.take(int(0.8 * len(ratings)))
ratings_val = shuffled.skip(int(0.8 * len(ratings))).take(int(0.2 * len(ratings)))

# Movies dataset
movies = pd.read_csv('recipe_dataset.csv')
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

# Select the basic features.
ratings_train = ratings_train.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
ratings_val = ratings_val.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
movies = movies.map(lambda x: x["Title"])

# Rest of the code remains the same...

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x:  tf.strings.as_string(x["User_id"])))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

class MovieLensModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.

    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval
    ):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates = movies.batch(128).map(movie_model)
    )
)

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# fitting model to ratings_train dataset
model.fit(ratings_train.batch(5000), epochs=20)

# evaluate model to ratings_val dataset
model.evaluate(ratings_val.batch(5000), return_dict=True)

Epoch 1/20
4/4 [==============================] - 4s 773ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 5.6644e-05 - factorized_top_k/top_10_categorical_accuracy: 1.1329e-04 - factorized_top_k/top_50_categorical_accuracy: 3.9651e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0014 - loss: 33924.4477 - regularization_loss: 0.0000e+00 - total_loss: 33924.4477
Epoch 2/20
4/4 [==============================] - 3s 756ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0065 - factorized_top_k/top_5_categorical_accuracy: 0.0411 - factorized_top_k/top_10_categorical_accuracy: 0.0709 - factorized_top_k/top_50_categorical_accuracy: 0.1740 - factorized_top_k/top_100_categorical_accuracy: 0.2345 - loss: 29638.8902 - regularization_loss: 0.0000e+00 - total_loss: 29638.8902
Epoch 3/20
4/4 [==============================] - 3s 722ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0544 - factorized_top_k/top_5_categoric

{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.0006798096583224833,
 'factorized_top_k/top_50_categorical_accuracy': 0.0038522547110915184,
 'factorized_top_k/top_100_categorical_accuracy': 0.006571493111550808,
 'loss': 60111.12890625,
 'regularization_loss': 0,
 'total_loss': 60111.12890625}

### **Second Optimization**

In [21]:
# Ratings data.
ratings = pd.read_csv('user_rating.csv')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

# Split into train and test set in rating dataset
tf.random.set_seed(42)
shuffled = ratings.shuffle(len(ratings), seed=42, reshuffle_each_iteration=False)
ratings_train = shuffled.take(int(0.7 * len(ratings)))
ratings_val = shuffled.skip(int(0.7 * len(ratings))).take(int(0.3 * len(ratings)))

# Movies dataset
movies = pd.read_csv('recipe_dataset.csv')
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

# Select the basic features.
ratings_train = ratings_train.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
ratings_val = ratings_val.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
movies = movies.map(lambda x: x["Title"])

# Rest of the code remains the same...

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: tf.strings.as_string(x["User_id"])))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

class MovieLensModel(tfrs.Model):
    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval
    ):
        super().__init__()

        self.user_model = user_model
        self.movie_model = movie_model
        self.task = task

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 128, embeddings_regularizer=tf.keras.regularizers.l2(0.01)),  # Increased embedding size
     # add l2 regulazation
    tf.keras.layers.Dropout(0.2) # Added dropout regularization
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 128, embeddings_regularizer=tf.keras.regularizers.l2(0.01)),  # Increased embedding size
    # add l2 regulazation
    tf.keras.layers.Dropout(0.2) # Added dropout regularization
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=movies.batch(128).map(movie_model)
    )
)

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# # Increase epochs for training
# model.fit(ratings_train.batch(5000), epochs=500)  # Increased number of epochs

# # Evaluate model on validation set
# model.evaluate(ratings_val.batch(5000), return_dict=True)

# Add early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_factorized_top_k/top_5_categorical_accuracy',
    patience=3,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights from the epoch with the lowest validation loss
)

# fitting model
model.fit(ratings_train.batch(100), validation_data=ratings_val.batch(100), epochs=500, callbacks=[early_stopping])

# Additional suggestions for experimentation:
# - Hyperparameter tuning
# - Data preprocessing techniques
# - Regularization techniques
# - Trying different loss functions
# - Ensemble methods


Epoch 1/500
155/155 [==============================] - 20s 128ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.9421e-04 - factorized_top_k/top_10_categorical_accuracy: 5.1790e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0022 - factorized_top_k/top_100_categorical_accuracy: 0.0058 - loss: 456.9317 - regularization_loss: 4.1366 - total_loss: 461.0683 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 3.0211e-04 - val_factorized_top_k/top_10_categorical_accuracy: 3.0211e-04 - val_factorized_top_k/top_50_categorical_accuracy: 0.0027 - val_factorized_top_k/top_100_categorical_accuracy: 0.0062 - val_loss: 59.9148 - val_regularization_loss: 0.2118 - val_total_loss: 60.1266
Epoch 2/500
155/155 [==============================] - 20s 130ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0030 - fact

## **User regulazation L2**

In [ ]:
# Ratings data.
ratings = pd.read_csv('user_rating.csv')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

# Split into train and test set in rating dataset
tf.random.set_seed(42)
shuffled = ratings.shuffle(len(ratings), seed=42, reshuffle_each_iteration=False)
ratings_train = shuffled.take(int(0.8 * len(ratings)))
ratings_val = shuffled.skip(int(0.8 * len(ratings))).take(int(0.2 * len(ratings)))

# Movies dataset
movies = pd.read_csv('recipe_dataset.csv')
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

# Select the basic features.
ratings_train = ratings_train.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
ratings_val = ratings_val.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
movies = movies.map(lambda x: x["Title"])

# Rest of the code remains the same...

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x:  tf.strings.as_string(x["User_id"])))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

class MovieLensModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.

    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval
    ):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64, embeddings_regularizer=tf.keras.regularizers.l2(0.01))
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64, embeddings_regularizer=tf.keras.regularizers.l2(0.01))
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates = movies.batch(128).map(movie_model)
    )
)

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# fitting model to ratings_train dataset
model.fit(ratings_train.batch(5000), epochs=10)

# evaluate model to ratings_val dataset
model.evaluate(ratings_val.batch(5000), return_dict=True)


Epoch 1/10
4/4 [==============================] - 40s 9s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 5.6644e-05 - factorized_top_k/top_10_categorical_accuracy: 5.6644e-05 - factorized_top_k/top_50_categorical_accuracy: 5.0980e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0011 - loss: 33928.6438 - regularization_loss: 61.5987 - total_loss: 33990.2422
Epoch 2/10
4/4 [==============================] - 41s 10s/step - factorized_top_k/top_1_categorical_accuracy: 0.0065 - factorized_top_k/top_5_categorical_accuracy: 0.0368 - factorized_top_k/top_10_categorical_accuracy: 0.0624 - factorized_top_k/top_50_categorical_accuracy: 0.1520 - factorized_top_k/top_100_categorical_accuracy: 0.2069 - loss: 30240.4160 - regularization_loss: 619.9056 - total_loss: 30860.3211
Epoch 3/10
4/4 [==============================] - 36s 9s/step - factorized_top_k/top_1_categorical_accuracy: 0.0490 - factorized_top_k/top_5_categorical_accurac

{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.0,
 'factorized_top_k/top_10_categorical_accuracy': 0.0006798096583224833,
 'factorized_top_k/top_50_categorical_accuracy': 0.0033990482334047556,
 'factorized_top_k/top_100_categorical_accuracy': 0.007477906066924334,
 'loss': 50115.67578125,
 'regularization_loss': 2242.15966796875,
 'total_loss': 52357.8359375}

## **Dropout Layers & early stopping**

In [ ]:
# Ratings data.
ratings = pd.read_csv('user_rating.csv')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

# Split into train and test set in rating dataset
tf.random.set_seed(42)
shuffled = ratings.shuffle(len(ratings), seed=42, reshuffle_each_iteration=False)
ratings_train = shuffled.take(int(0.8 * len(ratings)))
ratings_val = shuffled.skip(int(0.8 * len(ratings))).take(int(0.2 * len(ratings)))

# Movies dataset
movies = pd.read_csv('recipe_dataset.csv')
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

# Select the basic features.
ratings_train = ratings_train.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
ratings_val = ratings_val.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
movies = movies.map(lambda x: x["Title"])

# Rest of the code remains the same...

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x:  tf.strings.as_string(x["User_id"])))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

class MovieLensModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.

    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval
    ):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64, embeddings_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64, embeddings_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.2)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates = movies.batch(128).map(movie_model)
    )
)

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Add early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights from the epoch with the lowest validation loss
)

# fitting model
model.fit(ratings_train.batch(10_000), validation_data=ratings_val.batch(10_000), epochs=10, callbacks=[early_stopping])



Epoch 1/10
2/2 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 5.6644e-05 - factorized_top_k/top_100_categorical_accuracy: 5.0980e-04 - loss: 80277.4609 - regularization_loss: 23.0544 - total_loss: 80300.5156 

2/2 [==============================] - 56s 30s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 5.6644e-05 - factorized_top_k/top_100_categorical_accuracy: 5.0980e-04 - loss: 76335.6875 - regularization_loss: 27.2037 - total_loss: 76362.8906 - val_factorized_top_k/top_1_categorical_accuracy: 2.2660e-04 - val_factorized_top_k/top_5_categorical_accuracy: 9.0641e-04 - val_factorized_top_k/top_10_categorical_accuracy: 0.0016 - val_factorized_top_k/top_50_categorical_accuracy: 0.0029 - val_factorized_top_k/top_100_categorical_accuracy: 0.0066 - val_loss: 37052.5195 - val_regularization_loss: 102.1569 - val_total_loss: 37154.6758
Epoch 2/10
2/2 [==============================] - 49s 26s/step - factorized_top_k/top_1_categorical_accuracy: 2.8322e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0046 - factorized_t

## **Model complexity**
add dense layer and increase embedding layer

In [ ]:
# Ratings data.
ratings = pd.read_csv('user_rating.csv')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

# Split into train and test set in rating dataset
tf.random.set_seed(42)
shuffled = ratings.shuffle(len(ratings), seed=42, reshuffle_each_iteration=False)
ratings_train = shuffled.take(int(0.8 * len(ratings)))
ratings_val = shuffled.skip(int(0.8 * len(ratings))).take(int(0.2 * len(ratings)))

# Movies dataset
movies = pd.read_csv('recipe_dataset.csv')
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

# Select the basic features.
ratings_train = ratings_train.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
ratings_val = ratings_val.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
movies = movies.map(lambda x: x["Title"])

# Rest of the code remains the same...

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x:  tf.strings.as_string(x["User_id"])))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

class MovieLensModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.

    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval
    ):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 128, embeddings_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 128, embeddings_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates = movies.batch(128).map(movie_model)
    )
)

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Add early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights from the epoch with the lowest validation loss
)

# fitting model
model.fit(ratings_train.batch(10_000), validation_data=ratings_val.batch(10_000), epochs=10, callbacks=[early_stopping])



Epoch 1/10
2/2 [==============================] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 80306.1641 - regularization_loss: 24.4910 - total_loss: 80330.6562 

2/2 [==============================] - 46s 23s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 76373.8802 - regularization_loss: 25.5880 - total_loss: 76399.4688 - val_factorized_top_k/top_1_categorical_accuracy: 4.5321e-04 - val_factorized_top_k/top_5_categorical_accuracy: 4.5321e-04 - val_factorized_top_k/top_10_categorical_accuracy: 0.0014 - val_factorized_top_k/top_50_categorical_accuracy: 0.0057 - val_factorized_top_k/top_100_categorical_accuracy: 0.0084 - val_loss: 135155.6250 - val_regularization_loss: 100.6231 - val_total_loss: 135256.2500
Epoch 2/10
2/2 [==============================] - 43s 23s/step - factorized_top_k/top_1_categorical_accuracy: 1.6993e-04 - factorized_top_k/top_5_categorical_accuracy: 1.6993e-04 - factor

## **Change learning rate**
Change dropout

In [ ]:
# Ratings data.
ratings = pd.read_csv('user_rating.csv')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

# Split into train and test set in rating dataset
tf.random.set_seed(42)
shuffled = ratings.shuffle(len(ratings), seed=42, reshuffle_each_iteration=False)
ratings_train = shuffled.take(int(0.8 * len(ratings)))
ratings_val = shuffled.skip(int(0.8 * len(ratings))).take(int(0.2 * len(ratings)))

# Movies dataset
movies = pd.read_csv('recipe_dataset.csv')
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

# Select the basic features.
ratings_train = ratings_train.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
ratings_val = ratings_val.map(lambda x: {
    "movie_title": x["Title"],
    "user_id": tf.strings.as_string(x["User_id"])
})
movies = movies.map(lambda x: x["Title"])

# Rest of the code remains the same...

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x:  tf.strings.as_string(x["User_id"])))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

class MovieLensModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.

    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval
    ):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 132, embeddings_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.3)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 132, embeddings_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.3)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates = movies.batch(128).map(movie_model)
    )
)

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.01))

# Add early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_factorized_top_k/top_5_categorical_accuracy',
    patience=3,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights from the epoch with the lowest validation loss
)

# fitting model
model.fit(ratings_train.batch(10_000), validation_data=ratings_val.batch(10_000), epochs=10, callbacks=[early_stopping])

Epoch 1/10
2/2 [==============================] - 46s 24s/step - factorized_top_k/top_1_categorical_accuracy: 1.1329e-04 - factorized_top_k/top_5_categorical_accuracy: 2.8322e-04 - factorized_top_k/top_10_categorical_accuracy: 9.0631e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0033 - factorized_top_k/top_100_categorical_accuracy: 0.0067 - loss: 76334.0521 - regularization_loss: 21.8412 - total_loss: 76355.8958 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 6.7981e-04 - val_factorized_top_k/top_10_categorical_accuracy: 6.7981e-04 - val_factorized_top_k/top_50_categorical_accuracy: 0.0045 - val_factorized_top_k/top_100_categorical_accuracy: 0.0079 - val_loss: 37035.4805 - val_regularization_loss: 21.8084 - val_total_loss: 37057.2891
Epoch 2/10
2/2 [==============================] - 45s 24s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.6993e-04 - fa